Ich werde verschiedene Modelle trainieren und sie vergleichen.\
Kriterien sind:
- Dauer des Trainings
- Größe des Modells
- Genauigkeit in der Erkennung der verschiedenen Katzen

# YoloV11

# MobileNetV2

In [ ]:
#!pip install torch torchvision

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Von-0–1-Normalisierung + Resize
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std= [0.229,0.224,0.225])
])

train_ds = datasets.ImageFolder('data_split/train', transform=transform)
val_ds   = datasets.ImageFolder('data_split/val',   transform=transform)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4)
val_dl   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4)


In [ ]:
from torchvision.models import mobilenet_v2
import torch.nn as nn

# Vortrainiertes MobileNetV2
model = mobilenet_v2(pretrained=True)

# Finalen Klassifizierer auf 2 Klassen anpassen
model.classifier[1] = nn.Linear(model.last_channel, 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
best_acc = 0.0

for epoch in range(1, 21):  # z. B. 20 Epochen
    # --- Training ---
    model.train()
    running_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)

    # --- Validation ---
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
            total   += yb.size(0)
    acc = correct / total

    print(f"Epoch {epoch:02d}: Loss={(running_loss/len(train_ds)):.4f}, Val-Acc={acc:.4f}")

    # Speichere bestes Modell
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), 'catid_mobilenetv2.pt')

# EfficientNet-Lite0 